In [1]:
import open3d as o3d
import numpy as np
import rerun as rr

from icecream import ic
from pathlib import Path
from jaxtyping import UInt8, UInt16,Float32, Float64

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Visualize and Understand sample redwood rgbd dataset

In [2]:
# dataset_rgbd = o3d.data.SampleRedwoodRGBDImages()
dataset_rgbd = o3d.data.SampleFountainRGBDImages()
depth_path_list:list[str] = dataset_rgbd.depth_paths
image_path_list:list[str] = dataset_rgbd.color_paths
try:
    camera_trajectory = o3d.io.read_pinhole_camera_trajectory(
                          dataset_rgbd.odometry_log_path)
except:
    camera_trajectory = o3d.io.read_pinhole_camera_trajectory(
                          dataset_rgbd.keyframe_poses_log_path)
    

assert len(depth_path_list) == len(image_path_list) == len(camera_trajectory.parameters)
# ensure that intrinsics are the same for all images

intrinsics = camera_trajectory.parameters[0].intrinsic.intrinsic_matrix
assert all(np.allclose(intrinsics, cam_params.intrinsic.intrinsic_matrix) for cam_params in camera_trajectory.parameters)
K_33:Float32[np.ndarray, "3 3"] = intrinsics.astype(np.float32)

# Load and visualize rgbd images with given depth, image, odometry path, intri

In [3]:
import cv2
from tqdm.notebook import tqdm

def load_rgbd_images(redwood_rgbd:o3d.data.SampleRedwoodRGBDImages)-> list[o3d.geometry.RGBDImage]:
    rgbd_images = []
    depth_cv2_list = []
    for img_path, depth_path in zip(redwood_rgbd.color_paths, redwood_rgbd.depth_paths):
        depth_cv2 = cv2.imread(depth_path, cv2.IMREAD_UNCHANGED)
        depth_cv2_list.append(depth_cv2)
        depth = o3d.io.read_image(depth_path)
        depth = o3d.geometry.Image(depth_cv2)
        color = o3d.io.read_image(img_path)
        rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color=color, depth=depth, depth_scale=1000, depth_trunc=2.0 ,convert_rgb_to_intensity=False)
        rgbd_images.append(rgbd_image)
    return rgbd_images, depth_cv2_list

rr.init("Visualize RGBD")
parent_log_path = Path("world")
rr.log(f"{parent_log_path}", rr.ViewCoordinates.RDF, timeless=True)

rgbd_list, depth_cv2_list = load_rgbd_images(dataset_rgbd)
pbar = tqdm(enumerate(zip(rgbd_list, depth_cv2_list, camera_trajectory.parameters)), total=len(rgbd_list))
for timestep, (rgbd, depth_cv2, cam_params) in pbar:
    rr.set_time_sequence("timestep", timestep)
    img:UInt8[np.ndarray, "h w 3"] = np.asarray(rgbd.color)
    depth:UInt16[np.ndarray, "h w 3"] = np.asarray(rgbd.depth)

    cam_log_path = f"{parent_log_path}/cam"
    intrinsics = cam_params.intrinsic
    cam_T_world_44:Float64[np.ndarray, "4 4"] = cam_params.extrinsic
    rr.log(f"{cam_log_path}", rr.Transform3D(translation=cam_T_world_44[:3, 3], mat3x3=cam_T_world_44[:3, :3], from_parent=True))
    rr.log(f"{cam_log_path}/pinhole",
           rr.Pinhole(
               image_from_camera=intrinsics.intrinsic_matrix, 
               width=intrinsics.width,
               height=intrinsics.height,
               camera_xyz=rr.ViewCoordinates.RDF))
    rr.log(
        f"{cam_log_path}/pinhole/rgb",
        rr.Image(img)
    )
    rr.log(f"{cam_log_path}/pinhole/depth", rr.DepthImage(depth, meter=1.0))
    rr.log(f"{cam_log_path}/pinhole/depth_cv2", rr.DepthImage(depth_cv2))

rr.notebook_show()

## Perform RGBD Integration

In [ ]:

volume = o3d.pipelines.integration.ScalableTSDFVolume(
            voxel_length=4.0 / 512.0,
            sdf_trunc=0.04,
            color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8
        )

rr.init("TSDF Fusion RGBD")
parent_log_path = Path("world")
rr.log(f"{parent_log_path}", rr.ViewCoordinates.RDF, timeless=True)

rgbd_list, depth_cv2_list = load_rgbd_images(dataset_rgbd)
for timestep, (rgbd, cam_params) in enumerate(zip(rgbd_list, camera_trajectory.parameters)):
    rr.set_time_sequence("timestep", timestep)
    img:UInt8[np.ndarray, "h w 3"] = np.asarray(rgbd.color)
    depth:UInt16[np.ndarray, "h w 3"] = np.asarray(rgbd.depth)
    cam_log_path = f"{parent_log_path}/cam"
    intrinsics = cam_params.intrinsic
    cam_T_world_44:Float64[np.ndarray, "4 4"] = cam_params.extrinsic

    volume.integrate(
        image=rgbd,
        intrinsic=intrinsics,
        extrinsic=cam_T_world_44)


mesh = volume.extract_triangle_mesh()
mesh.compute_vertex_normals()

rr.log(f"{parent_log_path}/mesh", rr.Mesh3D(vertex_positions=mesh.vertices, indices=mesh.triangles, vertex_normals=mesh.vertex_normals, vertex_colors=mesh.vertex_colors))

rr.notebook_show()

# Try with Polycam Data

In [ ]:
cwd = Path.cwd()
polycam_data_dir = cwd.parent / "data"/"6G-bookcase-poly"
image_dir: Path = polycam_data_dir / "corrected_images"
camera_dir: Path = polycam_data_dir / "corrected_cameras"
depth_dir: Path | None = (
    polycam_data_dir / "depth" if (polycam_data_dir / "depth").exists() else None
    )

assert (
    image_dir.exists() and image_dir.is_dir()
), f"Image directory not found: {image_dir}"
assert (
    camera_dir.exists() and camera_dir.is_dir()
), f"Camera directory not found: {camera_dir}"

image_paths: list[Path] = sorted(image_dir.glob("*.jpg"))
camera_paths: list[Path] = sorted(camera_dir.glob("*.json"))
if depth_dir is not None:
    assert depth_dir.exists() and depth_dir.is_dir(), "Depth directory not found"
    depth_paths: list[Path] = sorted(depth_dir.glob("*.png"))
    assert len(image_paths) == len(depth_paths), "Image and depth mismatch"

In [ ]:
from PIL import Image
from monopriors.data.polycam_data import load_raw_polycam_data, PolycamCameraData
import calibur

volume = o3d.pipelines.integration.ScalableTSDFVolume(
            voxel_length=4.0 / 512.0,
            sdf_trunc=0.04,
            color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8
        )

rr.init("TSDF Fusion Polycam")
parent_log_path = Path("world")
rr.log(f"{parent_log_path}", rr.ViewCoordinates.LUB, timeless=True)

height, width, _ = np.array(Image.open(image_paths[0])).shape

pbar = tqdm(enumerate(zip(image_paths, camera_paths, depth_paths)), total=len(image_paths))
for idx, (image_path, camera_path, depth_path) in pbar:
    rr.set_time_sequence("idx", idx)
    rgb_hw3: UInt8[np.ndarray, "h w 3"] = np.array(Image.open(image_path))
    depth_hw = cv2.imread(str(depth_path), cv2.IMREAD_UNCHANGED) #/ 1000.0
    depth_hw: Float32[np.ndarray, "h w"] = cv2.resize(
                depth_hw,
                (width, height),
                interpolation=cv2.INTER_NEAREST,
            )
    cam_log_path = f"{parent_log_path}/cam"
    polycam_data:PolycamCameraData = load_raw_polycam_data(camera_path)

    K_33:Float32[np.ndarray, "3 3"] = np.array([
        [polycam_data.fx, 0, polycam_data.cx],
        [0, polycam_data.fy, polycam_data.cy],
        [0, 0, 1]
    ], dtype=np.float32)
    world_T_cam_44: Float32[np.ndarray, "4 4"] = np.array(
            [
                [polycam_data.t_00, polycam_data.t_01, polycam_data.t_02, polycam_data.t_03],
                [polycam_data.t_10, polycam_data.t_11, polycam_data.t_12, polycam_data.t_13],
                [polycam_data.t_20, polycam_data.t_21, polycam_data.t_22, polycam_data.t_23],
                [0, 0, 0, 1],
            ],
            dtype=np.float32,
        )
    
    world_T_cam_44 = calibur.convert_pose(
        world_T_cam_44, src_convention=calibur.CC.GL, dst_convention=calibur.CC.CV
    )
    cam_T_world_44 = np.linalg.inv(world_T_cam_44)

    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color=o3d.geometry.Image(rgb_hw3),
        depth=o3d.geometry.Image(depth_hw),
        depth_scale=1000.0,
        depth_trunc=3.0,
        convert_rgb_to_intensity=False
    )

    volume.integrate(
        image=rgbd,
        intrinsic=o3d.camera.PinholeCameraIntrinsic(
                width=width,
                height=height,
                fx=K_33[0, 0],
                fy=K_33[1, 1],
                cx=K_33[0, 2],
                cy=K_33[1, 2],
            ),
        extrinsic=cam_T_world_44)
    
    rr.log(f"{cam_log_path}", rr.Transform3D(translation=cam_T_world_44[:3, 3], mat3x3=cam_T_world_44[:3, :3], from_parent=True))
    rr.log(f"{cam_log_path}/pinhole",
           rr.Pinhole(
               image_from_camera=K_33, 
               width=polycam_data.width,
               height=polycam_data.height,
               camera_xyz=rr.ViewCoordinates.RDF))
    rr.log(
        f"{cam_log_path}/pinhole/rgb",
        rr.Image(rgb_hw3)
    )
    rr.log(
        f"{cam_log_path}/pinhole/depth",
        rr.DepthImage(depth_hw, meter=1000.0)
    )

mesh = volume.extract_triangle_mesh()
mesh.compute_vertex_normals()

rr.log(f"{parent_log_path}/mesh", rr.Mesh3D(vertex_positions=mesh.vertices, indices=mesh.triangles, vertex_normals=mesh.vertex_normals, vertex_colors=mesh.vertex_colors))

rr.notebook_show()
